In [1]:
!conda install --yes --quiet pymongo



# All requested packages already installed.
# packages in environment at /opt/conda:
#
pymongo                   3.4.0                    py36_0    defaults


In [2]:
%run __init__.py

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
%run mongo_conn.py

In [5]:
client = pymongo.MongoClient(mongo_ip, 27016)

db_text = 'wikipedia_text'
client.drop_database(db_text)
db_ref = client.wikipedia_text
coll_ref = db_ref.my_collection

MAX_DEPTH = 3

all_pages_dict = {}

In [6]:
def process_page(pg_id, p_immed, p_cat):
    pg_id = str(pg_id)
    pg_request = 'http://en.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&rvprop=content&pageids='+pg_id
    
    r = requests.get(pg_request)
    page_dict = r.json()['query']
       
    dict_key = pg_id+'/'+p_cat
    
    try:
        if dict_key not in all_pages_dict.keys():    
            all_pages_dict[dict_key] = {"page_title": page_dict['pages'][pg_id]['title'], 
                                        "page_text": page_dict['pages'][pg_id]['extract'], 
                                        'immed_parent': p_immed,
                                        'ult_parent': p_cat}
            
            coll_ref.insert_one(all_pages_dict[dict_key])
            
            
    except:
        with open("../error/problem_pages.txt", "a") as myfile:
            problem_page_str = pg_id+", parent: "+p_immed+"\n"
            myfile.write(problem_page_str)


In [7]:
def api_category(my_str):
    my_str_split = my_str.lstrip().rstrip().split(':')
    ret_str = my_str_split[0].lstrip().rstrip() + '%3A+'
    
    category_name = my_str_split[1].split()
    for element in category_name:
        ret_str += element+'+'
    
    return ret_str[:-1]

In [8]:
def process_category(my_cat, p_cat=None, depth_count=0):
    cat_action = 'query'
    cat_format = 'json'
    cat_list = 'categorymembers'

    cat_request  = 'http://en.wikipedia.org/w/api.php?action='+cat_action+'&'
    cat_request += 'format='+cat_format+'&'
    cat_request += 'list='+cat_list+'&'
    
    api_call = cat_request+'cmtitle='+api_category(my_cat)+'&cmlimit=max'

    r = requests.get(api_call)
    json_list = r.json()['query']['categorymembers']
    
    if p_cat == None:
        p_cat = my_cat
    
    depth_count += 1
    
    for row in json_list:
        if row['ns'] == 0:
            process_page(row['pageid'], my_cat, p_cat)
        elif row['ns'] == 14:
            if depth_count < MAX_DEPTH:
                process_category(row['title'], p_cat, depth_count)
                
    

In [9]:
my_cats = ['Category:Machine learning', 'Category:Business software']

for my_title in my_cats:
    process_category(my_title)

In [10]:
client.database_names(), db_ref.collection_names()

(['admin', 'local', 'test', 'wikipedia_text'], ['my_collection'])